In [1]:
# LIBRARIES

import pandas as pd
import numpy as np
from openpyxl import load_workbook
import xlsxwriter
import re

In [2]:
from translation_library import (column_names_translation, 
                                 shop_names_translation, 
                                 month_names_translation, 
                                 country_names_translation, 
                                 product_type_name_translation, 
                                 brand_name_translation, 
                                 shop_branch_mapping, 
                                 shop_location_mapping, 
                                 broad_categories, 
                                 dsitrict_mapping, 
                                 regions_mapping)

In [3]:
excel_files = ['2021', '2022 01-06', '2022 07-12', '2023 01-06','2023 07-12']

# correcting excel files
Excel Files were broken, pandas an other excel readig libraries did not read the file. This issue occures because of the speciic way that "1c" program renders excel files. 
below code is the correction opf that problem

In [4]:
import shutil
from zipfile import ZipFile
import os


def excel_correction_1c(filename):
    # Creating temporary folder
    tmp_folder = '/tmp/convert_wrong_excel/'
    os.makedirs(tmp_folder, exist_ok=True)

    # Extract excel as zip into our temporary folder
    with ZipFile(f'{filename}.xlsx') as excel_container:
        excel_container.extractall(tmp_folder)

    # Rename a file with an incorrect name
    wrong_file_path = os.path.join(tmp_folder, 'xl', 'SharedStrings.xml')
    correct_file_path = os.path.join(tmp_folder, 'xl', 'sharedStrings.xml')
    os.rename(wrong_file_path, correct_file_path) 

    # Wrap up excel back into zip and rename it to the original file
    shutil.make_archive(f'{filename}', 'zip', tmp_folder)
    os.rename(f'{filename}.zip', f'{filename}.xlsx')

In [ ]:
for file in excel_files:
    excel_correction_1c(file)

### Importing dataframes

In [4]:
DATA_PATH = '/Users/nairabarseghyan/Desktop/project_name/data/'

In [5]:
def reading_files(arr):
    ls = []
    for file in arr:
        df = pd.read_excel(f'{DATA_PATH}raw/{file}.xlsx', sheet_name = "TDSheet", engine="openpyxl", skiprows=11)
        ls.append(df)
    return ls

df2021, df2022_1, df2022_2, df2023_1, df2023_2 = reading_files(excel_files)
      

In [6]:
df_list = [df2021, df2022_1, df2022_2, df2023_1, df2023_2]

### Translating Dataframes

In [7]:
def df_preperation(arr):
    
    for dataframe in arr: #finding columns that are empty
        list = []
        for i in dataframe.columns:
            x = dataframe[f'{i}'].unique()
            if np.issubdtype(x.dtype, np.number) and np.isnan(x).all():
                list.append(i)
        dataframe.drop(list, axis=1,inplace=True)#droping empty columns
        dataframe.drop(dataframe.index[-1],inplace=True) #droping last row

In [8]:
def data_translation(arr):
    for dataframe in arr:
        dataframe.rename(columns=column_names_translation, inplace=True)
        dataframe['Shop'] = dataframe['Shop'].map(shop_names_translation).fillna(dataframe['Shop'])
        dataframe["sale_month"].replace(month_names_translation, regex=True, inplace=True)
        dataframe["order_month"].replace(month_names_translation, regex=True, inplace=True)
        dataframe["manufacture_country"].replace(country_names_translation, regex=True, inplace=True)
        #dataframe["product_type"].replace(product_type_name_translation, regex=True, inplace=True)
        dataframe['product_type'] = dataframe['product_type'].map(product_type_name_translation).fillna(dataframe['product_type'])
        dataframe['Brand'] = dataframe['Brand'].map(brand_name_translation).fillna(dataframe['Brand'])
        
        

In [9]:
df_preperation(df_list)

In [10]:
data_translation(df_list)

In [11]:
df2021.tail(2)

,Shop,vendor_code,sale_year,sale_month,sale_day,order_year,order_month,order_day,order_hour,order_minute,manufacture_country,product_type,Brand,Series,Quantity,Revenue+VAT
441091,Martuni/Yerevanyan_Nonoperating,ER-00010337,2021.0,April,16.0,2021.0,April,16.0,15.0,32.0,Italy,Mixer taps and parts,Teorema,Խառնիչ ծորակ Teo Sonic 8C15011 բաղ․,1.0,44080.0
441092,Martuni/Yerevanyan_Nonoperating,ER-00010771,2021.0,April,14.0,2021.0,April,12.0,12.0,16.0,Russia,Wallpaper washable,Palitra,Պաստառ P PL71358-29 (ֆլիզելին),4.0,48000.0


### Checking Translation

In [12]:
def has_non_english(text):
    if not isinstance(text, (str, bytes)):
        return False 
    non_english_pattern = re.compile('[^a-zA-Z0-9!@#$%^&*()_+={}|\[\]:;<>,.?/~`\s-]')
    if non_english_pattern.search(text):
        True
        n = text
        return n#bool(non_english_pattern.search(text))


In [13]:
def get_columns(dataframe):
    
    shop = dataframe['Shop'].unique().tolist()
    sale_month = dataframe['sale_month'].unique().tolist()
    order_month = dataframe['order_month'].unique().tolist()
    manufacture_country = dataframe['manufacture_country'].unique().tolist()
    product_type = dataframe['product_type'].unique().tolist()
    Brand = dataframe['Brand'].unique().tolist()
    
    column_lists = [shop, sale_month, order_month, manufacture_country, product_type, Brand]
    
    return column_lists

In [14]:
def non_english_checker(arr):

    results = []
    
    for df_index, dataframe in enumerate(arr):
        
        column_list = get_columns(dataframe)
        
        for column_index, column in enumerate(column_list):
            
            for string in column:
                if has_non_english(string):
                    results.append(("dataframe #", df_index, "column #", column_index, 'string', string ))
    return results

In [15]:
cehcker = non_english_checker(df_list)
cehcker

[('dataframe #', 0, 'column #', 4, 'string', 'Լadder'),
 ('dataframe #', 1, 'column #', 4, 'string', 'Լadder'),
 ('dataframe #', 2, 'column #', 4, 'string', 'Լadder'),
 ('dataframe #', 3, 'column #', 4, 'string', 'Լadder'),
 ('dataframe #', 3, 'column #', 5, 'string', "3 Day's"),
 ('dataframe #', 4, 'column #', 4, 'string', 'Լadder'),
 ('dataframe #', 4, 'column #', 5, 'string', "3 Day's")]

We see that there are no non-english(armenian) entires in all of our dataframes

### Adding columns

In [16]:
def shop_info(arr):
    for dataframe in arr:
        dataframe['Branch'] = dataframe['Shop'].map(shop_branch_mapping)
        dataframe['City'] = dataframe['Shop'].map(shop_location_mapping)
        dataframe['Districts'] = dataframe['Shop'].map(dsitrict_mapping)
        dataframe['Regions'] = dataframe['Shop'].map(regions_mapping)
        dataframe['Product_category'] = dataframe['product_type'].map(broad_categories)
        

In [17]:
shop_info(df_list)

### Dataframe Date processing 

In [18]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

In [19]:
def date_column(arr):
    for dataframne in arr:
        
        dataframne['sale_month'] = dataframne['sale_month'].map(month_mapping).astype(int)
        dataframne['sale_year'] = dataframne['sale_year'].astype(int)
        dataframne['sale_day'] = dataframne['sale_day'].astype(int)
        dataframne['sale_date'] = dataframne['sale_year'].astype(str) + '-' + dataframne['sale_month'].astype(str).str.zfill(2) + '-' + dataframne['sale_day'].astype(str).str.zfill(2)
        
        dataframne['order_month'] = dataframne['order_month'].map(month_mapping)
        dataframne['order_month'] = dataframne['order_month'].fillna(0).astype(int)
        dataframne['order_year'] = dataframne['order_year'].fillna(0).astype(int)
        dataframne['order_day'] = dataframne['order_day'].fillna(0).astype(int)
        dataframne['order_date'] = dataframne['order_year'].astype(str).str.zfill(4) + '-' + dataframne['order_month'].astype(str).str.zfill(2) + '-' + dataframne['order_day'].astype(str).str.zfill(2)
        dataframne['order_date'] = dataframne['order_date'].replace("0000-00-00", "UNK")
        
        
        dataframne['order_hour'] = dataframne['order_hour'].fillna(00).astype(int)
        dataframne['order_minute'] = dataframne['order_minute'].fillna(00).astype(int)
        dataframne['order_time'] = dataframne['order_hour'].astype(str).str.zfill(2) + ':' + dataframne['order_minute'].astype(str).str.zfill(2)
        dataframne['order_time'] = dataframne['order_time'].replace("00:00", "UNK")
        

In [20]:
date_column(df_list)

In [21]:
print('2021 date', df2021["order_date"].unique())

print('2022_1 date', df2022_1["order_date"].unique())

print('2022_2 date', df2022_2["order_date"].unique())

print('2023_1 date', df2023_1["order_date"].unique())

print('2023_2 date', df2023_2["order_date"].unique())

2021 date ['2021-03-25' '2021-09-20' '2021-10-31' '2021-02-13' '2021-04-14'
 '2021-05-15' '2021-10-30' '2021-12-05' '2021-04-05' '2021-08-02'
 '2021-08-23' '2021-12-10' '2021-12-20' '2021-02-18' '2021-03-04'
 '2021-03-19' '2021-03-27' '2021-04-07' '2021-04-30' '2021-05-13'
 '2021-05-23' '2021-06-15' '2021-06-25' '2021-06-28' '2021-09-30'
 '2021-09-29' '2021-11-03' '2021-11-05' '2021-12-16' '2021-12-17'
 '2021-12-18' '2021-12-23' '2021-12-27' '2021-09-15' '2021-10-10'
 '2021-03-21' '2021-03-23' '2021-10-16' '2021-04-02' '2021-12-08'
 '2021-08-26' '2021-09-19' '2021-12-24' '2021-05-03' '2021-07-19'
 '2021-07-25' '2021-08-14' '2021-08-20' '2021-08-18' '2021-11-26'
 '2021-05-16' '2021-06-07' '2021-08-24' '2021-09-23' '2021-10-11'
 '2021-06-19' '2021-08-12' '2021-08-19' '2021-11-14' '2021-12-06'
 '2021-12-07' '2021-02-24' '2021-06-27' '2021-10-27' '2021-08-13'
 '2021-08-21' '2021-10-25' '2021-09-03' '2021-11-15' '2021-11-18'
 '2021-10-20' '2021-02-07' '2021-08-30' '2021-03-18' '2021-02-11'


In [22]:
print('2021 time', df2021["order_time"].unique())

print('2022_1 time', df2022_1["order_time"].unique())

print('2022_2 time', df2022_2["order_time"].unique())

print('2023_1 time', df2023_1["order_time"].unique())

print('2023_2 time', df2023_2["order_time"].unique())

2021 time ['14:37' '20:09' '15:32' '12:06' '13:11' '17:50' '11:40' '13:07' '16:15'
 '13:21' '14:29' '13:47' '13:17' '16:30' '16:34' '14:53' '11:42' '17:47'
 '12:42' '11:56' '11:37' '13:26' '13:53' '16:58' '17:41' '18:39' '12:51'
 '17:43' '15:39' '16:10' '15:38' '19:05' '17:35' '12:12' '18:48' '11:33'
 '12:24' '12:01' '12:26' '13:31' '16:35' '13:49' '16:07' '18:50' '12:31'
 '16:49' '10:48' '16:59' '11:26' '11:35' '19:48' '10:45' '10:04' '14:32'
 '13:06' '13:05' '18:31' '16:25' '12:47' '15:27' '18:45' '17:33' '13:12'
 '17:55' '13:54' '18:23' '11:18' '17:30' '15:22' '16:55' '12:30' '12:04'
 '16:11' '16:06' '12:05' '18:44' '10:26' '13:15' '18:41' '10:50' '17:38'
 '14:15' '14:02' '18:58' '13:44' '14:39' '12:54' '10:10' '10:58' '18:55'
 '09:47' '11:21' '09:25' '16:57' '13:33' '13:09' '13:58' '13:13' '15:16'
 '11:07' '16:46' '14:40' '12:22' '14:55' '13:35' '17:09' '15:51' '19:57'
 '13:37' '14:20' '15:06' '17:18' '13:20' '14:25' '16:22' '13:34' '15:28'
 '17:27' '15:12' '10:17' '09:56' '18:12' 

In [23]:
def drop_cols(arr):
    for dataframe in arr:
        dataframe.drop(columns=['sale_day', 'sale_month', 'sale_year', 'order_day', 'order_month', 'order_year','order_hour','order_minute'], axis=1, inplace=True)

In [24]:
drop_cols(df_list)

### Adding Holiday column

In [25]:
holidays = pd.read_csv(f'{DATA_PATH}external/armenian_holidays.csv')

holidays['Date'] = pd.to_datetime(holidays['Date'])
holiday_dates = set(holidays['Date'])

def IsHoliday(arr):
    for dataframe in arr:
        dataframe['sale_date'] = pd.to_datetime(dataframe['sale_date'])
        dataframe['IsHoliday'] = dataframe['sale_date'].apply(lambda x: 1 if x in holiday_dates else 0)
        

In [26]:
IsHoliday(df_list)

In [27]:
df2021

,Shop,vendor_code,manufacture_country,product_type,Brand,Series,Quantity,Revenue+VAT,Branch,City,Districts,Regions,Product_category,sale_date,order_date,order_time,IsHoliday
0,01_Arshakunyats_shop,00-00000082,Iran,Tile floor,Persepolis,Սալիկ IR 60*60 Jupiter Grey (white body),25.00,97125.00,Arshakunyats S.,Yerevan,Shengavit,YEREVAN,Tiles,2021-03-31,2021-03-25,14:37,0
1,01_Arshakunyats_shop,00-00000082,Iran,Tile floor,Persepolis,Սալիկ IR 60*60 Jupiter Grey (white body),12.24,56125.82,Arshakunyats S.,Yerevan,Shengavit,YEREVAN,Tiles,2021-09-20,2021-09-20,20:09,0
2,01_Arshakunyats_shop,00-00000082,Iran,Tile floor,Persepolis,Սալիկ IR 60*60 Jupiter Grey (white body),0.36,1627.52,Arshakunyats S.,Yerevan,Shengavit,YEREVAN,Tiles,2021-10-31,2021-10-31,15:32,0
3,01_Arshakunyats_shop,00-00000086,China,Soap dish,Kaldo,Օճառաման Kaldo Soap dish,1.00,1276.39,Arshakunyats S.,Yerevan,Shengavit,YEREVAN,Sanceramics,2021-02-15,2021-02-13,12:06,0
4,01_Arshakunyats_shop,00-00000086,China,Soap dish,Kaldo,Օճառաման Kaldo Soap dish,1.00,1289.22,Arshakunyats S.,Yerevan,Shengavit,YEREVAN,Sanceramics,2021-04-20,2021-04-14,13:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441088,Martuni/Yerevanyan_Nonoperating,ER-00010133,China,Siphon parts,Kaldo,Սիֆոնի մետաղական գլխիկ Kaldo (C-004) chrome,1.00,2300.00,Martuni C.,Martuni,Other Regions,GEGHARKUNIK,Sanceramics,2021-05-10,2021-05-10,12:57,0
441089,Martuni/Yerevanyan_Nonoperating,ER-00010133,China,Siphon parts,Kaldo,Սիֆոնի մետաղական գլխիկ Kaldo (C-004) chrome,1.00,2880.00,Martuni C.,Martuni,Other Regions,GEGHARKUNIK,Sanceramics,2021-05-18,2021-05-18,15:30,0
441090,Martuni/Yerevanyan_Nonoperating,ER-00010333,NaN,Adhesive tapes and nets,NaN,Կպչուն ցանց շինարարական 50մ (10սմ),1.00,1900.00,Martuni C.,Martuni,Other Regions,GEGHARKUNIK,Other,2021-01-26,2021-01-26,14:06,0
441091,Martuni/Yerevanyan_Nonoperating,ER-00010337,Italy,Mixer taps and parts,Teorema,Խառնիչ ծորակ Teo Sonic 8C15011 բաղ․,1.00,44080.00,Martuni C.,Martuni,Other Regions,GEGHARKUNIK,Sanceramics,2021-04-16,2021-04-16,15:32,0


### Concatinating dictionaries together

In [29]:
df2022  = pd.concat([df2022_1, df2022_2])

In [30]:
df2022.shape

(545813, 17)

In [31]:
df2023 = pd.concat([df2023_1, df2023_2])

In [32]:
df2023.shape

(578239, 17)

In [33]:
full_df = pd.concat([df2021, df2022, df2023])

In [34]:
full_df.shape

(1565145, 17)

### Saving dataframes as csv and excel files

In [35]:
df2021.to_csv(f'{DATA_PATH}processed/sales2021.csv', index=False) 
df2022.to_csv(f'{DATA_PATH}processed/sales2022.csv', index=False) 
df2023.to_csv(f'{DATA_PATH}processed/sales2023.csv', index=False) 
full_df.to_csv(f'{DATA_PATH}processed/sales_whole.csv', index=False) 

In [36]:
df2021.to_excel(f'{DATA_PATH}processed/xlsx/sales2021.xlsx', index=False, engine='xlsxwriter')
df2022_1.to_excel(f'{DATA_PATH}processed/xlsx/sales2022_1.xlsx', index=False, engine='xlsxwriter')
df2022_2.to_excel(f'{DATA_PATH}processed/xlsx/sales2022_2.xlsx', index=False, engine='xlsxwriter')
df2023_1.to_excel(f'{DATA_PATH}processed/xlsx/sales2023_1.xlsx', index=False, engine='xlsxwriter')
df2023_2.to_excel(f'{DATA_PATH}processed/xlsx/sales2023_2.xlsx', index=False, engine='xlsxwriter')

# END